In [ ]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Parameters
img_width, img_height = 128, 128
batch_size = 32
epochs = 50

# Directories
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Data generator for training data

# Data generator for training data with validation split
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=40,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)  # 20% validation split

# Separate generator for training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')  # Specify 'training' subset

# Separate generator for validation data
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')  # Specify 'validation' subset



# Data generator for test data
test_files = [os.path.join(test_dir, f) for f in os.listdir(test_dir) if os.path.isfile(os.path.join(test_dir, f))]

# Create a DataFrame with the filenames
test_df = pd.DataFrame({"Filename": test_files})

test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='Filename',
    y_col=None,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False
)


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Load pre-trained MobileNetV2 model without including fully connected layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze most layers of the pre-trained model
for layer in base_model.layers[:-5]:
    layer.trainable = False

# Add custom layers on top of the pre-trained base model
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)  # num_classes is the number of your output classes

# Combine the base model with custom dense layers
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model (fine-tuning)
model.fit(train_generator, epochs=10, validation_data=validation_generator)


Downloading data from https://storage.googleapis.com/tensorflow/keras-applications/mobilenet_v2/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5
9406464/9406464 [==============================] - 3s 0us/step
Epoch 1/10
248/248 [==============================] - 32s 122ms/step - loss: 0.7014 - accuracy: 0.7467 - val_loss: 2.0278 - val_accuracy: 0.5879
Epoch 2/10
248/248 [==============================] - 30s 123ms/step - loss: 0.5522 - accuracy: 0.7968 - val_loss: 3.6277 - val_accuracy: 0.4106
Epoch 3/10
248/248 [==============================] - 29s 118ms/step - loss: 0.4918 - accuracy: 0.8254 - val_loss: 3.3398 - val_accuracy: 0.5258
Epoch 4/10
248/248 [==============================] - 31s 123ms/step - loss: 0.4393 - accuracy: 0.8390 - val_loss: 1.8787 - val_accuracy: 0.6040
Epoch 5/10
248/248 [==============================] - 29s 117ms/step - loss: 0.4090 - accuracy: 0.8523 - val_loss: 1.6951 - val_accuracy: 0.6687
Epoch 6/10
248/248 [==============================] - 30s 119ms/step - loss: 0.3731 - accuracy: 0.8620 - val_loss: 2.1868 - val_accuracy: 0.6293
Epoch 7/10
248/248 [==============================] - 29s 118ms/step - loss: 0.3641 - accuracy: 0.8660 - val_loss: 2.4951 - val_accuracy: 0.5313
Epoch 8/10
248/248 [==============================] - 30s 120ms/step - loss: 0.3443 - accuracy: 0.8788 - val_loss: 2.3415 - val_accuracy: 0.6278
Epoch 9/10
248/248 [==============================] - 30s 120ms/step - loss: 0.3242 - accuracy: 0.8818 - val_loss: 2.0466 - val_accuracy: 0.6768
Epoch 10/10
248/248 [==============================] - 33s 132ms/step - loss: 0.3180 - accuracy: 0.8838 - val_loss: 1.2926 - val_accuracy: 0.6556
